In [3]:
import numpy as np
import pandas as pd
import matplotlib as plt
import yfinance as yf
import keras 
import os
import datetime 
from sklearn.preprocessing import MinMaxScaler

In [41]:
def TickerIntoTensor(ticker='aapl',daysago=100,daysforward=1,split_percentage=0.9):
    df = yf.Ticker(ticker.upper())
    df = df.history(period='max')
    if df.shape[1] == 7:
        df = df.drop(columns=['Dividends','Stock Splits'])
    df = df.drop(columns=['Open','High','Low','Volume'])
    for i in range(daysago,-daysforward-1,-1):
        df['{}daysago'.format(i)] = df['Close'].shift(i)
        df = df.copy()
    df = df.drop(columns=['Close'])
    df = df.drop(index=df.tail(daysforward).index)
    df = df.drop(index=df.head(daysago).index)
    dfstddev = pd.DataFrame()
    dfstddev['daysago# std'] = df.iloc[:,:daysago+1].std(axis=1)
    dfstddev['half of daysago# std'] = df.iloc[:,daysago//2:daysago+1].std(axis=1)
    dfstddev['third of daysago# std'] = df.iloc[:,daysago-(daysago//3):daysago+1].std(axis=1)
    dfstddev['quarter of daysago# std'] = df.iloc[:,daysago-(daysago//4):daysago+1].std(axis=1)
    dfstddev['ten days from daysago# std'] = df.iloc[:,daysago-10:daysago+1].std(axis=1)
    print(dfstddev)
    sc = MinMaxScaler(feature_range=(0,1))
    stddev_scaled = sc.fit_transform(dfstddev)
    print(stddev_scaled)
    df_scaled = sc.fit_transform(df)
    X,Y = df_scaled[:,:daysago+1],df_scaled[:,-daysforward:]
    if split_percentage <= 1 and split_percentage >= 0:
        split = int(len(X)*split_percentage)
    else:
        split = split_percentage
    X_train, X_test, Y_train, Y_test = X[:split], X[split:], Y[:split], Y[split:]
    X_train, X_test = np.append(stddev_scaled[:split],X_train,axis=1), np.append(stddev_scaled[split:],X_test,axis=1)
    X_train, X_test, Y_train, Y_test = X_train.reshape((-1,daysago+1+dfstddev.shape[1],1)), X_test.reshape((-1,daysago+1+dfstddev.shape[1],1)), Y_train.reshape((-1,daysforward)), Y_test.reshape((-1,daysforward))

    return X_train, X_test, Y_train, Y_test
TickerIntoTensor('aapl',100,1,0.9)


                           daysago# std  half of daysago# std  \
Date                                                            
1981-05-07 00:00:00-04:00      0.010798              0.006353   
1981-05-08 00:00:00-04:00      0.010794              0.006396   
1981-05-11 00:00:00-04:00      0.010794              0.006412   
1981-05-12 00:00:00-04:00      0.010756              0.006433   
1981-05-13 00:00:00-04:00      0.010735              0.006448   
...                                 ...                   ...   
2024-06-27 00:00:00-04:00     13.642688             15.600956   
2024-06-28 00:00:00-04:00     13.915165             15.608001   
2024-07-01 00:00:00-04:00     14.304257             15.709450   
2024-07-02 00:00:00-04:00     14.755505             15.838103   
2024-07-03 00:00:00-04:00     15.217496             15.869533   

                           third of daysago# std  quarter of daysago# std  \
Date                                                                        


(array([[[4.08300022e-04],
         [2.74263105e-04],
         [2.83831532e-04],
         ...,
         [2.72269772e-04],
         [2.56185156e-04],
         [2.60537327e-04]],
 
        [[4.08081869e-04],
         [2.76936856e-04],
         [2.82266036e-04],
         ...,
         [2.60346312e-04],
         [2.62051582e-04],
         [2.64425612e-04]],
 
        [[4.08033650e-04],
         [2.77939116e-04],
         [2.82385698e-04],
         ...,
         [2.66308025e-04],
         [2.65962465e-04],
         [2.54704800e-04]],
 
        ...,
 
        [[4.15155246e-01],
         [2.33298178e-01],
         [2.81118926e-01],
         ...,
         [3.35563837e-01],
         [3.19708046e-01],
         [3.32612417e-01]],
 
        [[4.07894214e-01],
         [2.27662565e-01],
         [2.88185933e-01],
         ...,
         [3.24900990e-01],
         [3.34545575e-01],
         [3.21817844e-01]],
 
        [[4.00955859e-01],
         [2.22774682e-01],
         [2.97423652e-01],
         